In [1]:
import os
import re
import praw
import datetime as dt
from psaw import PushshiftAPI
from collections import defaultdict as ddict

In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient()
db = client.cmv
posts_collection = db.posts
tl_comments_collection = db.tl_comments
deltad_replies_collection = db.deltad_replies

In [4]:
#db.posts.drop()
#db.tl_comments.drop()
#db.deltad_replies.drop()

In [5]:
api = PushshiftAPI()

In [6]:
reddit = praw.Reddit(client_id='_h0r1eMioK_N3g', client_secret=os.environ['red_sec'],
                    password=os.environ['red_pas'], user_agent='X11; Ubuntu; Linux x86_64; rv:61.0:ThePersuasionEquation:v0.1',
                     username='OfAquitaine')
#subreddit = reddit.subreddit('changemyview')

In [7]:
#used date ranges:
# 1/1/2016 - 1/1/2017
start_epoch = int(dt.datetime(2017, 1, 1).timestamp())
end_epoch = int(dt.datetime(2018, 1, 1).timestamp())
# time_now = dt.datetime.now()

In [8]:
post_gen = api.search_submissions(subreddit='changemyview',
                                  after=start_epoch,
                                  before=end_epoch,
                                )

In [9]:
delta_types = ['!delta', 'Δ', '&#8710;']

In [10]:
footnote_to_filter_out = r"""
> *Hello, users of CMV! This is a footnote from your moderators. We'd just like to remind you of a couple of things. Firstly, please remember to* ***[read through our rules](http://www.reddit.com/r/changemyview/wiki/rules)***. *If you see a comment that has broken one, it is more effective to report it than downvote it. Speaking of which,* ***[downvotes don't change views](http://www.reddit.com/r/changemyview/wiki/guidelines#wiki_upvoting.2Fdownvoting)****! If you are thinking about submitting a CMV yourself, please have a look through our* ***[popular topics wiki](http://www.reddit.com/r/changemyview/wiki/populartopics)*** *first. Any questions or concerns? Feel free to* ***[message us](http://www.reddit.com/message/compose?to=/r/changemyview)***. *Happy CMVing!*
"""
footnote_regex = re.compile('\>\s\*Hello, users of CMV!.*\*Happy CMVing!\*')

In [11]:
c = 0
for post in post_gen:
    try:
        if post.author== '[deleted]':
            continue
        if post.selftext == '[removed]':
            continue
        if post.num_comments == 0:
            continue
    except:
        continue
    
    c += 1
    if c % 50 == 0:
        print(c)
        
    #     comment_gen = api._get_submission_comment_ids(submission_id=post.id)
    post_dict = ddict(lambda: [])
    post_dict['post_id'] = post.id
    post_dict['post_title'] = post.title
    post_dict['post_author'] = str(post.author)

    # filter out CMV subreddit footnote that often appears in posts:
    post_text = post.selftext
    post_text = re.sub(footnote_regex,"",post_text)

    post_dict['post_text'] = post_text

    #connect to reddit API with PRAW
    praw_post = reddit.submission(id=post.id)
    # using replace_more to fill instances of "load more comments" with the actual comments.
    praw_post.comments.replace_more()

    for tl_comment in praw_post.comments:
        if tl_comment.body == '[deleted]':
            continue
        
        post_dict['comment_ids'].append(tl_comment.id)

        tl_comment_dict = ddict(lambda: [])
        tl_comment_dict['comment_id'] = tl_comment.id
        tl_comment_dict['comment_text'] = tl_comment.body
        tl_comment_dict['author'] = str(tl_comment.author)


        # using replace_more to fill instances of "load more comments" with the actual comments.
        tl_comment.replies.replace_more()

        num_replies = 0
        # list() method flattens comment tree.
        all_replies = tl_comment.replies.list()
        for reply_num, reply in enumerate(all_replies):
            num_replies += 1

            # check to see if author wrote response: we only care about op-awarded deltas.
            if str(reply.author) == post_dict['post_author']:
                op_reply = reply.body

                # check to see if delta is awarded in op's reply
                if any(deltastring in op_reply for deltastring in delta_types):
                    print('found delta string')
                    post_dict['tl_comment_delta_parents'].append(tl_comment.id)

                    # if op's delta award was not in response to a top-level comment, save info about the lower-level comment that OP awarded the delta to.
                    if reply_num == 0:
                        post_dict['deltad_tl_comments'].append(tl_comment.id)
                    else:
                        deltad_reply = all_replies[reply_num-1]
                        post_dict['deltad_replies'].append(deltad_reply.id)
                        tl_comment_dict['deltad_reply_ids'].append(deltad_reply.id)
                        deltad_reply_dict = {'reply_id': deltad_reply.id,
                                             'comment_text': deltad_reply.body,
                                             'author': str(deltad_reply.author),
                                             'tl_comment_id': tl_comment.id,
                                             'depth': reply_num-1}
                        # insert deltad reply dict into MongoDB collection of (sub-level) deltad replies
                        deltad_replies_collection.insert_one(deltad_reply_dict)

        # insert top level comment into MongoDB collection of tl comments
        tl_comments_collection.insert_one(tl_comment_dict)

    # insert post dict into MongoDB collection of posts
    try:
        posts_collection.insert_one(post_dict)
    except Exception as error:
        print('entering except statement: {}'.format(str(error)))

found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
50
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found delta string
found del

In [147]:
# posts_collection.aggregate([
#     { "$match": { "deltad_tl_comments": { "$ne": []} } },
#     { "$group": "post_id" }
# ])